<a href="https://colab.research.google.com/github/BooEiEi/project/blob/main/imputation_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn import svm
import sys
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
!pip install missingpy
import missingpy
from missingpy import MissForest
from sklearn.metrics import mean_squared_error
from sklearn import neighbors
from sklearn.impute import KNNImputer
import numpy as np
import random

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import os
from google.colab import drive #เชื่อม Google drive
drive.mount('/content/drive')
path = '/content/drive/My Drive/project' #เข้า drive
data = pd.read_csv(os.path.join(path,'df123.csv'))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
data.drop('Unnamed: 0',1,inplace = True)
data

<ipython-input-3-ec119c09c27b>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data.drop('Unnamed: 0',1,inplace = True)


,Sex,AgeSYear,telomere length (kb),Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,...,003_Creatinine,004_Uric Acid,005_Cholesterol,006_Triglyceride,008_HDL-C,009_LDL,010_AST,012_ALT,013_Alkaline Phos,071_HbA1c
0,2,58.0,7.30,47.2,156.0,19.4,112.0,64.0,70.0,NaN,...,0.6,4.8,180.0,97.0,62.0,98.0,25.0,32.0,40.0,5.8
1,2,57.0,6.36,47.4,153.0,20.2,114.0,75.0,69.0,19.3,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.1
2,2,24.0,9.71,53.2,157.0,21.6,103.0,69.0,78.0,21.3,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.4
3,2,62.0,6.17,54.4,165.0,20.0,95.0,75.0,83.0,21.6,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.6
4,2,86.0,4.55,49.0,154.0,20.7,129.0,79.0,83.0,NaN,...,0.7,5.4,183.0,86.0,39.0,127.0,12.0,22.0,82.0,6.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417,2,25.0,8.98,47.0,165.0,17.3,104.0,63.0,89.0,NaN,...,0.6,4.8,232.0,36.0,96.0,128.0,18.0,12.0,46.0,NaN
418,2,31.0,8.23,72.2,168.0,25.6,123.0,89.0,69.0,NaN,...,0.6,6.1,187.0,49.0,51.0,127.0,21.0,24.0,71.0,NaN
419,2,32.0,8.79,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.7,5.1,222.0,58.0,79.0,132.0,17.0,10.0,34.0,NaN
420,1,29.0,8.93,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,6.8,140.0,92.0,48.0,73.0,31.0,75.0,69.0,NaN


In [4]:
telomere = data.loc[:,['telomere length (kb) ']]

In [5]:
data.drop('telomere length (kb) ',1,inplace = True)

<ipython-input-5-bc57154f22a4>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data.drop('telomere length (kb) ',1,inplace = True)


In [6]:
data.isna().sum()

Sex                          0
AgeSYear                     0
Weight                       8
Height                       8
BMI                          8
Systolic                     8
Diastolic                    9
HeartRate                   51
SMM                        225
Fat Mass                   225
% Body fat                 225
waist to hip radio         225
abdominal circumference    226
visceral fat area          225
001_Glucose                  1
002_BUN                      0
003_Creatinine               0
004_Uric Acid               93
005_Cholesterol              0
006_Triglyceride             0
008_HDL-C                    0
009_LDL                      0
010_AST                      0
012_ALT                      1
013_Alkaline Phos           29
071_HbA1c                  167
dtype: int64

In [7]:
data1 = data.copy()
data2 = data.copy()
data3 = data.copy()
data4 = data.copy()

## data.iloc[:, 2:5] by MiCE-LR imputation
LR = LinearRegression model

In [8]:
data1 = data1.drop(data.columns[5: 15], axis=1)
data1 = data1.drop(data.columns[23: 26], axis=1)
data1 = data1.drop(columns=['004_Uric Acid'])
data1

,Sex,AgeSYear,Weight,Height,BMI,002_BUN,003_Creatinine,005_Cholesterol,006_Triglyceride,008_HDL-C,009_LDL,010_AST
0,2,58.0,47.2,156.0,19.4,10.0,0.6,180.0,97.0,62.0,98.0,25.0
1,2,57.0,47.4,153.0,20.2,15.0,1.0,176.0,51.0,72.0,94.0,16.0
2,2,24.0,53.2,157.0,21.6,12.0,0.8,159.0,45.0,52.0,98.0,13.0
3,2,62.0,54.4,165.0,20.0,17.0,0.7,230.0,94.0,61.0,151.0,18.0
4,2,86.0,49.0,154.0,20.7,9.0,0.7,183.0,86.0,39.0,127.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...
417,2,25.0,47.0,165.0,17.3,15.0,0.6,232.0,36.0,96.0,128.0,18.0
418,2,31.0,72.2,168.0,25.6,15.0,0.6,187.0,49.0,51.0,127.0,21.0
419,2,32.0,NaN,NaN,NaN,10.0,0.7,222.0,58.0,79.0,132.0,17.0
420,1,29.0,NaN,NaN,NaN,23.0,1.0,140.0,92.0,48.0,73.0,31.0


In [9]:
data1.isna().sum()

Sex                 0
AgeSYear            0
Weight              8
Height              8
BMI                 8
002_BUN             0
003_Creatinine      0
005_Cholesterol     0
006_Triglyceride    0
008_HDL-C           0
009_LDL             0
010_AST             0
dtype: int64

In [10]:
lr = LinearRegression()
imp = IterativeImputer(estimator= lr , max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean')
data1 = imp.fit_transform(data1)

[IterativeImputer] Completing matrix with shape (422, 12)
[IterativeImputer] Ending imputation round 1/10, elapsed time 0.01
[IterativeImputer] Change: 1.00794954514188, scaled tolerance: 0.396 
[IterativeImputer] Ending imputation round 2/10, elapsed time 0.02
[IterativeImputer] Change: 0.5630699514711139, scaled tolerance: 0.396 
[IterativeImputer] Ending imputation round 3/10, elapsed time 0.04
[IterativeImputer] Change: 0.5381585324036813, scaled tolerance: 0.396 
[IterativeImputer] Ending imputation round 4/10, elapsed time 0.05
[IterativeImputer] Change: 0.519920826208601, scaled tolerance: 0.396 
[IterativeImputer] Ending imputation round 5/10, elapsed time 0.06
[IterativeImputer] Change: 0.5045274901435626, scaled tolerance: 0.396 
[IterativeImputer] Ending imputation round 6/10, elapsed time 0.07
[IterativeImputer] Change: 0.4896287100332586, scaled tolerance: 0.396 
[IterativeImputer] Ending imputation round 7/10, elapsed time 0.08
[IterativeImputer] Change: 0.475207911177097

/usr/local/lib/python3.8/dist-packages/sklearn/impute/_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [11]:
data1

array([[  2.        ,  58.        ,  47.2       , ...,  62.        ,
         98.        ,  25.        ],
       [  2.        ,  57.        ,  47.4       , ...,  72.        ,
         94.        ,  16.        ],
       [  2.        ,  24.        ,  53.2       , ...,  52.        ,
         98.        ,  13.        ],
       ...,
       [  2.        ,  32.        ,  57.99150818, ...,  79.        ,
        132.        ,  17.        ],
       [  1.        ,  29.        ,  59.30275313, ...,  48.        ,
         73.        ,  31.        ],
       [  2.        ,  31.        ,  58.64547739, ...,  87.        ,
        124.        ,  24.        ]])

In [12]:
data1= pd.DataFrame(data1)
data1= data1.rename(columns={0: 'Sex' ,1: 'AgeSYear', 2:'Weight',3:'Height',4:'BMI',5:'BUN',6:'Creatinine',7:'Cholesterol',
                                8:'Triglyceride',9:'HDL-C',10:'LDL',11:'AST'})

data1

,Sex,AgeSYear,Weight,Height,BMI,BUN,Creatinine,Cholesterol,Triglyceride,HDL-C,LDL,AST
0,2.0,58.0,47.200000,156.000000,19.400000,10.0,0.6,180.0,97.0,62.0,98.0,25.0
1,2.0,57.0,47.400000,153.000000,20.200000,15.0,1.0,176.0,51.0,72.0,94.0,16.0
2,2.0,24.0,53.200000,157.000000,21.600000,12.0,0.8,159.0,45.0,52.0,98.0,13.0
3,2.0,62.0,54.400000,165.000000,20.000000,17.0,0.7,230.0,94.0,61.0,151.0,18.0
4,2.0,86.0,49.000000,154.000000,20.700000,9.0,0.7,183.0,86.0,39.0,127.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...
417,2.0,25.0,47.000000,165.000000,17.300000,15.0,0.6,232.0,36.0,96.0,128.0,18.0
418,2.0,31.0,72.200000,168.000000,25.600000,15.0,0.6,187.0,49.0,51.0,127.0,21.0
419,2.0,32.0,57.991508,161.301621,22.181490,10.0,0.7,222.0,58.0,79.0,132.0,17.0
420,1.0,29.0,59.302753,164.941868,21.593093,23.0,1.0,140.0,92.0,48.0,73.0,31.0


In [13]:
data2.iloc[:,0:5] = data1.iloc[:,0:5]

In [14]:
data2

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,003_Creatinine,004_Uric Acid,005_Cholesterol,006_Triglyceride,008_HDL-C,009_LDL,010_AST,012_ALT,013_Alkaline Phos,071_HbA1c
0,2.0,58.0,47.200000,156.000000,19.400000,112.0,64.0,70.0,NaN,NaN,...,0.6,4.8,180.0,97.0,62.0,98.0,25.0,32.0,40.0,5.8
1,2.0,57.0,47.400000,153.000000,20.200000,114.0,75.0,69.0,19.3,12.5,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.1
2,2.0,24.0,53.200000,157.000000,21.600000,103.0,69.0,78.0,21.3,14.6,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.4
3,2.0,62.0,54.400000,165.000000,20.000000,95.0,75.0,83.0,21.6,15.2,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.6
4,2.0,86.0,49.000000,154.000000,20.700000,129.0,79.0,83.0,NaN,NaN,...,0.7,5.4,183.0,86.0,39.0,127.0,12.0,22.0,82.0,6.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417,2.0,25.0,47.000000,165.000000,17.300000,104.0,63.0,89.0,NaN,NaN,...,0.6,4.8,232.0,36.0,96.0,128.0,18.0,12.0,46.0,NaN
418,2.0,31.0,72.200000,168.000000,25.600000,123.0,89.0,69.0,NaN,NaN,...,0.6,6.1,187.0,49.0,51.0,127.0,21.0,24.0,71.0,NaN
419,2.0,32.0,57.991508,161.301621,22.181490,NaN,NaN,NaN,NaN,NaN,...,0.7,5.1,222.0,58.0,79.0,132.0,17.0,10.0,34.0,NaN
420,1.0,29.0,59.302753,164.941868,21.593093,NaN,NaN,NaN,NaN,NaN,...,1.0,6.8,140.0,92.0,48.0,73.0,31.0,75.0,69.0,NaN


In [15]:
data2.isna().sum()

Sex                          0
AgeSYear                     0
Weight                       0
Height                       0
BMI                          0
Systolic                     8
Diastolic                    9
HeartRate                   51
SMM                        225
Fat Mass                   225
% Body fat                 225
waist to hip radio         225
abdominal circumference    226
visceral fat area          225
001_Glucose                  1
002_BUN                      0
003_Creatinine               0
004_Uric Acid               93
005_Cholesterol              0
006_Triglyceride             0
008_HDL-C                    0
009_LDL                      0
010_AST                      0
012_ALT                      1
013_Alkaline Phos           29
071_HbA1c                  167
dtype: int64

## data.iloc[:,5:24] by Miss Forest imputation

In [ ]:
list1 = []
for i in data2:
  random.seed(99)
  if i == '013_Alkaline Phos':
    break
  else:
    list1.append(i)
    MF = MissForest()
    MF_imput = MF.fit_transform(data2[list1].copy())

In [17]:
data2= pd.DataFrame(MF_imput)
data2 = data2.rename(columns={0: 'Sex' ,1: 'AgeSYear', 2:'Weight',3:'Height',4:'BMI',5:'Systolic',6:'Diastolic',7:'HeartRate',8:'SMM',9:'Fat Mass',
                                10:'% Body fat',11:'waist to hip radio',12:'abdominal circumference',13:'visceral fat area',14:'Glucose',15:'BUN',16:'Creatinine',17:'Uric Acid',18:'Cholesterol',
                                19:'Triglyceride',20:'HDL-C',21:'LDL',22:'AST',23:'ALT'})

data2

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,Glucose,BUN,Creatinine,Uric Acid,Cholesterol,Triglyceride,HDL-C,LDL,AST,ALT
0,2.0,58.0,47.200000,156.000000,19.400000,112.000,64.00,70.00,19.741,13.631,...,100.0,10.0,0.6,4.8,180.0,97.0,62.0,98.0,25.0,32.0
1,2.0,57.0,47.400000,153.000000,20.200000,114.000,75.00,69.00,19.300,12.500,...,81.0,15.0,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0
2,2.0,24.0,53.200000,157.000000,21.600000,103.000,69.00,78.00,21.300,14.600,...,79.0,12.0,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0
3,2.0,62.0,54.400000,165.000000,20.000000,95.000,75.00,83.00,21.600,15.200,...,89.0,17.0,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0
4,2.0,86.0,49.000000,154.000000,20.700000,129.000,79.00,83.00,19.907,15.369,...,108.0,9.0,0.7,5.4,183.0,86.0,39.0,127.0,12.0,22.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417,2.0,25.0,47.000000,165.000000,17.300000,104.000,63.00,89.00,20.387,12.852,...,82.0,15.0,0.6,4.8,232.0,36.0,96.0,128.0,18.0,12.0
418,2.0,31.0,72.200000,168.000000,25.600000,123.000,89.00,69.00,25.451,18.258,...,85.0,15.0,0.6,6.1,187.0,49.0,51.0,127.0,21.0,24.0
419,2.0,32.0,57.991508,161.301621,22.181490,113.160,71.83,72.30,22.707,21.599,...,94.0,10.0,0.7,5.1,222.0,58.0,79.0,132.0,17.0,10.0
420,1.0,29.0,59.302753,164.941868,21.593093,108.465,71.96,83.95,27.607,20.258,...,80.0,23.0,1.0,6.8,140.0,92.0,48.0,73.0,31.0,75.0


In [18]:
data3.iloc[:,5:24] = data2.iloc[:,5:24]

In [19]:
data3

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,003_Creatinine,004_Uric Acid,005_Cholesterol,006_Triglyceride,008_HDL-C,009_LDL,010_AST,012_ALT,013_Alkaline Phos,071_HbA1c
0,2,58.0,47.2,156.0,19.4,112.000,64.00,70.00,19.741,13.631,...,0.6,4.8,180.0,97.0,62.0,98.0,25.0,32.0,40.0,5.8
1,2,57.0,47.4,153.0,20.2,114.000,75.00,69.00,19.300,12.500,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.1
2,2,24.0,53.2,157.0,21.6,103.000,69.00,78.00,21.300,14.600,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.4
3,2,62.0,54.4,165.0,20.0,95.000,75.00,83.00,21.600,15.200,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.6
4,2,86.0,49.0,154.0,20.7,129.000,79.00,83.00,19.907,15.369,...,0.7,5.4,183.0,86.0,39.0,127.0,12.0,22.0,82.0,6.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417,2,25.0,47.0,165.0,17.3,104.000,63.00,89.00,20.387,12.852,...,0.6,4.8,232.0,36.0,96.0,128.0,18.0,12.0,46.0,NaN
418,2,31.0,72.2,168.0,25.6,123.000,89.00,69.00,25.451,18.258,...,0.6,6.1,187.0,49.0,51.0,127.0,21.0,24.0,71.0,NaN
419,2,32.0,NaN,NaN,NaN,113.160,71.83,72.30,22.707,21.599,...,0.7,5.1,222.0,58.0,79.0,132.0,17.0,10.0,34.0,NaN
420,1,29.0,NaN,NaN,NaN,108.465,71.96,83.95,27.607,20.258,...,1.0,6.8,140.0,92.0,48.0,73.0,31.0,75.0,69.0,NaN


In [20]:
data3.isna().sum()

Sex                          0
AgeSYear                     0
Weight                       8
Height                       8
BMI                          8
Systolic                     0
Diastolic                    0
HeartRate                    0
SMM                          0
Fat Mass                     0
% Body fat                   0
waist to hip radio           0
abdominal circumference      0
visceral fat area            0
001_Glucose                  0
002_BUN                      0
003_Creatinine               0
004_Uric Acid                0
005_Cholesterol              0
006_Triglyceride             0
008_HDL-C                    0
009_LDL                      0
010_AST                      0
012_ALT                      0
013_Alkaline Phos           29
071_HbA1c                  167
dtype: int64

## data.iloc[:, 24:26] by KNN imputation

In [21]:
list1 = []
for i in data3:
  random.seed(99)
  if i == '013_Alkaline Phos':
    list1.append(i)
    KNN_imputer = KNNImputer(n_neighbors = 15)
    KNN_imput = KNN_imputer.fit_transform(data3[list1].copy())
  elif i == '071_HbA1c':
    list1.append(i)
    KNN_imputer = KNNImputer(n_neighbors = 15)
    KNN_imput = KNN_imputer.fit_transform(data3[list1].copy())
  else:
    list1.append(i)

In [22]:
dataset = pd.DataFrame(KNN_imput)
dataset = dataset.rename(columns={0: 'Sex' ,1: 'AgeSYear', 2:'Weight',3:'Height',4:'BMI',5:'Systolic',6:'Diastolic',7:'HeartRate',8:'SMM',9:'Fat Mass',
                                10:'% Body fat',11:'waist to hip radio',12:'abdominal circumference',13:'visceral fat area',14:'Glucose',15:'BUN',16:'Creatinine',17:'Uric Acid',18:'Cholesterol',
                                19:'Triglyceride',20:'HDL-C',21:'LDL',22:'AST',23:'ALT',24:'Alkaline Phos',25:'HbA1c'})

dataset.round(4)

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,Creatinine,Uric Acid,Cholesterol,Triglyceride,HDL-C,LDL,AST,ALT,Alkaline Phos,HbA1c
0,2.0,58.0,47.2000,156.0000,19.40,112.000,64.00,70.00,19.741,13.631,...,0.6,4.8,180.0,97.0,62.0,98.0,25.0,32.0,40.0,5.8000
1,2.0,57.0,47.4000,153.0000,20.20,114.000,75.00,69.00,19.300,12.500,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.1000
2,2.0,24.0,53.2000,157.0000,21.60,103.000,69.00,78.00,21.300,14.600,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.4000
3,2.0,62.0,54.4000,165.0000,20.00,95.000,75.00,83.00,21.600,15.200,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.6000
4,2.0,86.0,49.0000,154.0000,20.70,129.000,79.00,83.00,19.907,15.369,...,0.7,5.4,183.0,86.0,39.0,127.0,12.0,22.0,82.0,6.2000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417,2.0,25.0,47.0000,165.0000,17.30,104.000,63.00,89.00,20.387,12.852,...,0.6,4.8,232.0,36.0,96.0,128.0,18.0,12.0,46.0,5.3933
418,2.0,31.0,72.2000,168.0000,25.60,123.000,89.00,69.00,25.451,18.258,...,0.6,6.1,187.0,49.0,51.0,127.0,21.0,24.0,71.0,5.1867
419,2.0,32.0,54.5000,158.4667,21.68,113.160,71.83,72.30,22.707,21.599,...,0.7,5.1,222.0,58.0,79.0,132.0,17.0,10.0,34.0,5.4867
420,1.0,29.0,63.0733,168.8667,22.04,108.465,71.96,83.95,27.607,20.258,...,1.0,6.8,140.0,92.0,48.0,73.0,31.0,75.0,69.0,4.9733


In [23]:
dataset.isna().sum()

Sex                        0
AgeSYear                   0
Weight                     0
Height                     0
BMI                        0
Systolic                   0
Diastolic                  0
HeartRate                  0
SMM                        0
Fat Mass                   0
% Body fat                 0
waist to hip radio         0
abdominal circumference    0
visceral fat area          0
Glucose                    0
BUN                        0
Creatinine                 0
Uric Acid                  0
Cholesterol                0
Triglyceride               0
HDL-C                      0
LDL                        0
AST                        0
ALT                        0
Alkaline Phos              0
HbA1c                      0
dtype: int64

In [25]:
result = dataset.join(telomere)
result

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,Uric Acid,Cholesterol,Triglyceride,HDL-C,LDL,AST,ALT,Alkaline Phos,HbA1c,telomere length (kb)
0,2.0,58.0,47.200000,156.000000,19.40,112.000,64.00,70.00,19.741,13.631,...,4.8,180.0,97.0,62.0,98.0,25.0,32.0,40.0,5.800000,7.30
1,2.0,57.0,47.400000,153.000000,20.20,114.000,75.00,69.00,19.300,12.500,...,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.100000,6.36
2,2.0,24.0,53.200000,157.000000,21.60,103.000,69.00,78.00,21.300,14.600,...,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.400000,9.71
3,2.0,62.0,54.400000,165.000000,20.00,95.000,75.00,83.00,21.600,15.200,...,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.600000,6.17
4,2.0,86.0,49.000000,154.000000,20.70,129.000,79.00,83.00,19.907,15.369,...,5.4,183.0,86.0,39.0,127.0,12.0,22.0,82.0,6.200000,4.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417,2.0,25.0,47.000000,165.000000,17.30,104.000,63.00,89.00,20.387,12.852,...,4.8,232.0,36.0,96.0,128.0,18.0,12.0,46.0,5.393333,8.98
418,2.0,31.0,72.200000,168.000000,25.60,123.000,89.00,69.00,25.451,18.258,...,6.1,187.0,49.0,51.0,127.0,21.0,24.0,71.0,5.186667,8.23
419,2.0,32.0,54.500000,158.466667,21.68,113.160,71.83,72.30,22.707,21.599,...,5.1,222.0,58.0,79.0,132.0,17.0,10.0,34.0,5.486667,8.79
420,1.0,29.0,63.073333,168.866667,22.04,108.465,71.96,83.95,27.607,20.258,...,6.8,140.0,92.0,48.0,73.0,31.0,75.0,69.0,4.973333,8.93


In [26]:
#result.to_csv(os.path.join(path,'df_1.csv'))

# เติมพร้อมกันทุกตัวแปร

In [ ]:
    MF = MissForest()
    MF_imput = MF.fit_transform(data4)

In [29]:
dataset1 = pd.DataFrame(MF_imput)
dataset1 = dataset1.rename(columns={0: 'Sex' ,1: 'AgeSYear', 2:'Weight',3:'Height',4:'BMI',5:'Systolic',6:'Diastolic',7:'HeartRate',8:'SMM',9:'Fat Mass',
                                10:'% Body fat',11:'waist to hip radio',12:'abdominal circumference',13:'visceral fat area',14:'Glucose',15:'BUN',16:'Creatinine',17:'Uric Acid',18:'Cholesterol',
                                19:'Triglyceride',20:'HDL-C',21:'LDL',22:'AST',23:'ALT',24:'Alkaline Phos',25:'HbA1c'})

dataset1.round(4)

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,Creatinine,Uric Acid,Cholesterol,Triglyceride,HDL-C,LDL,AST,ALT,Alkaline Phos,HbA1c
0,2.0,58.0,47.200,156.00,19.40,112.000,64.00,70.00,19.694,13.404,...,0.6,4.8,180.0,97.0,62.0,98.0,25.0,32.0,40.0,5.800
1,2.0,57.0,47.400,153.00,20.20,114.000,75.00,69.00,19.300,12.500,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.100
2,2.0,24.0,53.200,157.00,21.60,103.000,69.00,78.00,21.300,14.600,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.400
3,2.0,62.0,54.400,165.00,20.00,95.000,75.00,83.00,21.600,15.200,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.600
4,2.0,86.0,49.000,154.00,20.70,129.000,79.00,83.00,20.057,16.295,...,0.7,5.4,183.0,86.0,39.0,127.0,12.0,22.0,82.0,6.200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417,2.0,25.0,47.000,165.00,17.30,104.000,63.00,89.00,20.299,13.578,...,0.6,4.8,232.0,36.0,96.0,128.0,18.0,12.0,46.0,5.316
418,2.0,31.0,72.200,168.00,25.60,123.000,89.00,69.00,25.286,18.181,...,0.6,6.1,187.0,49.0,51.0,127.0,21.0,24.0,71.0,5.215
419,2.0,32.0,65.656,156.96,25.98,115.860,73.17,74.72,23.573,24.765,...,0.7,5.1,222.0,58.0,79.0,132.0,17.0,10.0,34.0,5.542
420,1.0,29.0,67.768,169.90,23.87,117.205,75.50,85.37,29.753,16.638,...,1.0,6.8,140.0,92.0,48.0,73.0,31.0,75.0,69.0,5.725


In [30]:
result1 = dataset1.join(telomere)
result1

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,Uric Acid,Cholesterol,Triglyceride,HDL-C,LDL,AST,ALT,Alkaline Phos,HbA1c,telomere length (kb)
0,2.0,58.0,47.200,156.00,19.40,112.000,64.00,70.00,19.694,13.404,...,4.8,180.0,97.0,62.0,98.0,25.0,32.0,40.0,5.800,7.30
1,2.0,57.0,47.400,153.00,20.20,114.000,75.00,69.00,19.300,12.500,...,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.100,6.36
2,2.0,24.0,53.200,157.00,21.60,103.000,69.00,78.00,21.300,14.600,...,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.400,9.71
3,2.0,62.0,54.400,165.00,20.00,95.000,75.00,83.00,21.600,15.200,...,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.600,6.17
4,2.0,86.0,49.000,154.00,20.70,129.000,79.00,83.00,20.057,16.295,...,5.4,183.0,86.0,39.0,127.0,12.0,22.0,82.0,6.200,4.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417,2.0,25.0,47.000,165.00,17.30,104.000,63.00,89.00,20.299,13.578,...,4.8,232.0,36.0,96.0,128.0,18.0,12.0,46.0,5.316,8.98
418,2.0,31.0,72.200,168.00,25.60,123.000,89.00,69.00,25.286,18.181,...,6.1,187.0,49.0,51.0,127.0,21.0,24.0,71.0,5.215,8.23
419,2.0,32.0,65.656,156.96,25.98,115.860,73.17,74.72,23.573,24.765,...,5.1,222.0,58.0,79.0,132.0,17.0,10.0,34.0,5.542,8.79
420,1.0,29.0,67.768,169.90,23.87,117.205,75.50,85.37,29.753,16.638,...,6.8,140.0,92.0,48.0,73.0,31.0,75.0,69.0,5.725,8.93


In [31]:
#result1.to_csv(os.path.join(path,'df_2.csv'))